# Define Classes for Field and Term Objects


In [140]:
import numpy as np

In [141]:
class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_index(self):
        return self.index
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq
    
class term(object):
    def __init__(self, field_list, lorentz_contractions=[], spinor_contractions=[]):
        self.field_list = field_list #ordered list of field objects
        self.lorentz_contractions = lorentz_contractions
        self.spinor_contractions = spinor_contractions #list of 2-tuples, indicating index values of fields in self.fields that are contracted 
    def __eq__(self, other):
        eq = (self.field_list == other.field_list)&(self.lorentz_contractions == other.lorentz_contractions)&(self.spinor_contractions == other.spinor_contractions)
        return eq
    def __str__(self):
        return 'fields: ' + str([item.get_symbol() for item in self.field_list]) + '\n' \
            + 'contractions: ' + str(self.contractions)
    def get_field_list(self):
        return self.field_list
    def get_lorentz_contractions(self):
        return self.lorentz_contractions
    def get_spinor_contractions(self):
        return self.spinor_contractions
    def set_field_list(self, field_list):
        self.field_list = field_list
    def set_lorentz_contractions(self, lorentz_contractions):
        self.lorentz_contractions = lorentz_contractions
    def set_spinor_contractions(self, spinor_contractions):
        self.spinor_contractions = spinor_contractions    
    def massDim(self):
        massDim = 0
        field_list = self.field_list
        #print(field_list)
        for field in field_list:
            massDim += field.get_massDim()
        return massDim 
    def get_field_symbols(self):
        field_symbols = []
        for item in self.field_list:
            field_symbols.append(item.get_symbol())
        return field_symbols    

In [142]:
#spacetime derivative
D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)

#dynamical fields
F = field('A', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)

#16 spinor space blinears: 1 lorentz scalar, 4 lorentz vectors, 6 lorentz tensors, 4 lorentz pseudovector, 1 lorentz pseudoscalar. 
#not dynamical fields, constant matrices in spinor space
_S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
_V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
_Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these

# Generate All Terms of a Given Mass Dimension - with One Free Lorentz Index (for IBP), and with No Free Indices

*Preliminary thoughts*: Fix mass dimension, M. Fields in the term must be such that mass dimensions add up to M. Note that Dirac bilinear terms have no mass dimension, so the mass dimension does not directly constrain the allowed number of these. But constant matrices relate different index spaces (e.g., lorentz, spinor). 
- count number of derivatives up to and including 0, ..., M-1 (only M-1 and not M because a term must contain at least one dynamical field). 
- for each number of derivatives, find all combinations of fields consistent with the mass dimension (forgetting about contractions for the moment).
- for each number of derivatives, and for each such combination of fields, perform all possible contractions, including all combinations of constant bilinear spinor space matrices. 
- define an equivalence relation between terms (based on commutation/anticommutation of derivatives/fields, various identities such as Fierz identities) to remove repeats. 

In [143]:
def generate_field_combos_d(massDim, num_derivs):
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=1.5, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0) #Psi
    Pb = field('Pb', massDim=1.5, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1) #Psi_bar
    
    if massDim <= num_derivs:
        return []
    else:
        combo_root = num_derivs*[D]
        combo_roots_list = [combo_root]
        massDim_res_max = massDim - num_derivs
       
        while massDim_res_max >= 2:
            combo_roots_list_new = []

            #print("")
            #print("combo_roots_list:")
            #for combo in combo_roots_list:
                #print(str([item.get_symbol() for item in combo]))
        
            for i in range(len(combo_roots_list)):
                massDim_root = sum([item.get_massDim() for item in combo_roots_list[i]])
                massDim_res = massDim - massDim_root
                #append any elements that already have residual massDim less than 2
                if massDim_res < 2:
                    combo_roots_list_new.append(combo_roots_list[i])
                if massDim_res >= 2:
                    combo_root_new1 = combo_roots_list[i].copy()
                    combo_root_new1.append(F)
                    combo_roots_list_new.append(combo_root_new1)
                if massDim_res >= 3:
                    combo_root_new2 = combo_roots_list[i].copy()
                    combo_root_new2.append(Pb)
                    combo_root_new2.append(P)
                    combo_roots_list_new.append(combo_root_new2)  
            #for combo in combo_roots_list_new:
                #print(str([item.get_symbol() for item in combo]))
            
            massDim_res_max = max([(massDim - sum([item.get_massDim() for item in combo_roots_list_new[i]])) for i in range(len(combo_roots_list_new))])
            #print("massDim_res_max: " + str(massDim_res_max))
            combo_roots_list = combo_roots_list_new
        
        #prune combo_roots_list, removing any terms with mass dimension massDim-1
        #print("")
        #print("combo_roots_list_final:")
        #for combo in combo_roots_list:
            #print(str([item.get_symbol() for item in combo]))
            
        combos_list_pruned = []
        for combo in combo_roots_list:
            combo_massDim = sum([item.get_massDim() for item in combo])
            if combo_massDim == massDim:
                combos_list_pruned.append(combo)
            else:
                pass    

    return combos_list_pruned

In [144]:
field_combos = generate_field_combos_d(10, 4)
print("len(field_combos): " + str(len(field_combos)))
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 2
['D', 'D', 'D', 'D', 'F', 'F', 'F']
['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P']


In [145]:
def generate_field_combos(massDim):
    combos_list = []
    for num_derivs in range(massDim):
        combos_list_d = generate_field_combos_d(massDim, num_derivs)
        combos_list += combos_list_d
    return combos_list        

In [146]:
field_combos = generate_field_combos(10)
print("len(field_combos): " + str(len(field_combos)))
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 26
['F', 'F', 'F', 'F', 'F']
['F', 'F', 'Pb', 'P', 'Pb', 'P']
['F', 'Pb', 'P', 'F', 'Pb', 'P']
['F', 'Pb', 'P', 'Pb', 'P', 'F']
['Pb', 'P', 'F', 'F', 'Pb', 'P']
['Pb', 'P', 'F', 'Pb', 'P', 'F']
['Pb', 'P', 'Pb', 'P', 'F', 'F']
['D', 'F', 'F', 'F', 'Pb', 'P']
['D', 'F', 'F', 'Pb', 'P', 'F']
['D', 'F', 'Pb', 'P', 'F', 'F']
['D', 'Pb', 'P', 'F', 'F', 'F']
['D', 'Pb', 'P', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'F', 'F', 'F', 'F']
['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'Pb', 'P', 'F', 'Pb', 'P']
['D', 'D', 'Pb', 'P', 'Pb', 'P', 'F']
['D', 'D', 'D', 'F', 'F', 'Pb', 'P']
['D', 'D', 'D', 'F', 'Pb', 'P', 'F']
['D', 'D', 'D', 'Pb', 'P', 'F', 'F']
['D', 'D', 'D', 'D', 'F', 'F', 'F']
['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'Pb', 'P', 'F']
['D', 'D', 'D', 'D', 'D', 'D', 'F', 'F']
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'F']


In [147]:
def convert_to_symbol_list(field_list):
    symbol_list = [item.get_symbol() for item in field_list]
    return(symbol_list)

def field_counts(symbol_list):
    D_count = 0
    F_count = 0
    PPb_count = 0 #Pb always comes with P, so no need to count Pb
    for item in symbol_list:
        if item == 'D':
            D_count += 1
        elif item == 'F':
            F_count += 1
        elif item == 'P':
            PPb_count += 1 
    field_counts = [D_count, F_count, PPb_count]
    return field_counts

def combos_equiv(symbol_list1, symbol_list2):
    return field_counts(symbol_list1) == field_counts(symbol_list2)



In [148]:
field_counts(convert_to_symbol_list(field_combos[11]))

[1, 0, 3]

In [149]:
combos_equiv(convert_to_symbol_list(field_combos[10]), convert_to_symbol_list(field_combos[11]))

False

In [171]:
def reduce_field_combos(combo_list):
    #INPUT
    #combo_list: list of lists of field symbols (strings)
    #OUTPUT
    #combo_list_reduced: list of lists of field symbols, with repeat combos removed
    #EXPLANATION: removes repeat field combos from combo_list
    #print("len(combo_list): " + str(len(combo_list)))
    combo_list_reduced = [combo_list[0]]
    for i in range(len(combo_list)):
        #print("combo1: " + str(combo1))
        #print("")
        #print("i: " + str(i))
        #print("len(combo_list_reduced): " + str(len(combo_list_reduced)))
        for j in range(len(combo_list_reduced)):
            #print("combo2: " + str(combo2))
            #print("j: " + str(j))
            if combos_equiv(combo_list[i], combo_list_reduced[j]):
                #print("break")
                break
        else:
            combo_list_reduced.append(combo_list[i])
    return combo_list_reduced  
    
    

In [172]:
massDim = 10
symbol_lists = []
for combo in generate_field_combos(massDim):
    symbol_lists.append(convert_to_symbol_list(combo))
symbol_lists
    
combo_list_reduced = reduce_field_combos(symbol_lists)
len(combo_list_reduced)
combo_list_reduced

[['F', 'F', 'F', 'F', 'F'],
 ['F', 'F', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'F', 'F', 'F', 'Pb', 'P'],
 ['D', 'Pb', 'P', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'F', 'F', 'F', 'F'],
 ['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'D', 'F', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'F', 'F', 'F'],
 ['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'F', 'F'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'F']]

In [173]:
#helper functions for generate_derivative_assignments
def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def sums_to_n(n, m): # generates partitions of derivatives to each field
    #INPUT
    #n: number we want to sum to; i.e. total number of derivatives
    #m: number of digits in sum; i.e. total number of fields
    #OUTPUT
    #list of lists of digits, with each sublist no longer than m
    #EXPLANATION
    #list of combos for n can be generated from list of combos for n-1 either by appending 1 to each of the combos
    #for n-1, or adding 1 to just one of the elements in each combo. To ensure no repeats, we require the lists to be
    #non-increasing.
    combos_list = []
    if n==0:
        combo = m*[0]
        combos_list.append(combo)
        return combos_list
    elif n==1:
        combo = [1] + (m-1)*[0]
        combos_list.append(combo)
        return combos_list
    else:
        combos_list_prev = sums_to_n(n-1, n-1)
        for combo in combos_list_prev:
            combo_new1 = combo + [1]
            if non_increasing(combo_new1) and combo_new1 not in combos_list:
                combos_list.append(combo_new1) 
            for i in range(len(combo)):
                combo_new2 = combo.copy()
                combo_new2[i] += 1
                if non_increasing(combo_new2) and combo_new2 not in combos_list:
                    combos_list.append(combo_new2) 
        combos_list_trunc = [combo for combo in combos_list if len(combo) <= m]    
        return combos_list_trunc

def pad_w_zeros(list_of_lists, size):
    #INPUT
    #list_of_lists: list of lists of numbers; all sublists should have length less than or equal to size
    #size: int, common size of all lists after padding
    #OUTPUT
    #list_of_lists_padded: padded list of lists, where each sublist has length size
    
    if size < max([len(sublist) for sublist in list_of_lists]):
        print("'size' input must be larger than or equal max length of sublists in 'list_of_lists' input.")
        return None
    else:
        list_of_lists_padded = []
        for sublist in list_of_lists:
            num_zeros = size - len(sublist)
            sublist_padded = sublist + num_zeros*[0]
            list_of_lists_padded.append(sublist_padded)
        return list_of_lists_padded
        
def sums_to_n_padded(n, m):
    return pad_w_zeros(sums_to_n(n, m), m)
    
    

def generate_derivative_assignments(field_combo):
    #INPUT
    #field_combo: List of string symbols for derivatives and non-derivative fields (called simply fields here).  
    #OUTPUT
    #derivative_assigments: list of lists, where each sublist contains a distinct assignment of derivatives in
    #field_combo to fields in field_combo
    #EXPLANATION: generates all assignments of derivatives to fields for a given combination. 
    d = 0 #initialize number of derivatives
    n_F = 0 #initialize number of F fields
    n_P = 0 #initialize number of P fields
    derivative_assigment_list = [] #lists of lists, where the elements of each sublist are field symbols; the 
    #number of 'D' symbols preceding a given non-derivative field indicates the number of derivatives acting on 
    #that field.
    for item in field_combo:
        if item == 'D':
            d += 1
        elif item == 'F':
            n_F += 1
        elif item == 'P':
            n_P += 1
        #no need to count Pb since there are the same number of these as of P
    
    #field lists for insertion of derivatives
    F_fields = n_F*['F']
    P_fields = n_P*['P']
    Pb_fields = n_P*['Pb']
    
    for d_F in range(d+1): #loop over total number of derivatives acting on F fields
        for d_P in range(d-d_F+1): #loop over total number of derivatives acting on P fields
            d_Pb = d-d_F-d_P #loop over total number of derivatives acting on Pb fields
            #calculate distribution of derivatives separately for each field type
            #print('d: ' + str(d))
            #print('d_F: ' + str(d_F))
            #print('d_P: ' + str(d_P))
            #print('d_Pb: ' + str(d_Pb))
            derivs_F = sums_to_n_padded(d_F, n_F)  
            derivs_P = sums_to_n_padded(d_P, n_P)
            derivs_Pb = sums_to_n_padded(d_Pb, n_P)

            for num_derivs_list_F in derivs_F:
                for num_derivs_list_P in derivs_P:
                    for num_derivs_list_Pb in derivs_Pb:
                        #build up different derivative assignments
                        deriv_assignment = []

                        #print('num_derivs_list_F:' + str(num_derivs_list_F))
                        #print('num_derivs_list_P:' + str(num_derivs_list_P))
                        #print('num_derivs_list_Pb:' + str(num_derivs_list_Pb))

                        for n_d in num_derivs_list_F: 
                            derivs = n_d*['D']
                            deriv_assignment += derivs + ['F']
                        for n_d in num_derivs_list_P: 
                            derivs = n_d*['D']
                            deriv_assignment += derivs + ['Pb']
                        for n_d in num_derivs_list_Pb: 
                            derivs = n_d*['D']
                            deriv_assignment += derivs + ['P']

                        #print('deriv_assignment: ' + str(deriv_assignment))
                        derivative_assigment_list.append(deriv_assignment)
                        
    return derivative_assigment_list
              
    

In [174]:
generate_derivative_assignments(['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P'])

[['F', 'Pb', 'D', 'D', 'D', 'D', 'D', 'P'],
 ['F', 'D', 'Pb', 'D', 'D', 'D', 'D', 'P'],
 ['F', 'D', 'D', 'Pb', 'D', 'D', 'D', 'P'],
 ['F', 'D', 'D', 'D', 'Pb', 'D', 'D', 'P'],
 ['F', 'D', 'D', 'D', 'D', 'Pb', 'D', 'P'],
 ['F', 'D', 'D', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'F', 'Pb', 'D', 'D', 'D', 'D', 'P'],
 ['D', 'F', 'D', 'Pb', 'D', 'D', 'D', 'P'],
 ['D', 'F', 'D', 'D', 'Pb', 'D', 'D', 'P'],
 ['D', 'F', 'D', 'D', 'D', 'Pb', 'D', 'P'],
 ['D', 'F', 'D', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'F', 'Pb', 'D', 'D', 'D', 'P'],
 ['D', 'D', 'F', 'D', 'Pb', 'D', 'D', 'P'],
 ['D', 'D', 'F', 'D', 'D', 'Pb', 'D', 'P'],
 ['D', 'D', 'F', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'F', 'Pb', 'D', 'D', 'P'],
 ['D', 'D', 'D', 'F', 'D', 'Pb', 'D', 'P'],
 ['D', 'D', 'D', 'F', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'F', 'Pb', 'D', 'P'],
 ['D', 'D', 'D', 'D', 'F', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P']]

In [175]:


def generate_fully_contracted(massDim):
    return term_list
            
def generate_IBP_vectors(deriv_assignments, num_free_lorentz):
    #EXPLANATION: generates all operators with one free lorentz index (i.e., lorentz vectors) for the purpose of
    #generating IBP relations. 
    return terms_contracted            
        